# 2. Geospatial Data Formats and IO
## 2.1 Geospatial Data Models
#### 2.1.1 Vector Data Model
Vector geospatial data has commonly been broken down into points, lines, and polygons to model real world entities. In some cases the type of model to use is obvious. In other cases it is not. For example, depending on the spatial scale you are conducting analysis a house could be modeled as a point or a polygon. Let's take a closer look:

* Points
    * Large-scale (fine)
        * events
        * fire hydrants
        * wells
    * Small-scale (broad)
        * cities
        * pollution source
        * centroids

<img src="./img/burlingame_centroids.png" width="600" height="300"/></img>

* Lines
    * Roads
    * Rivers
    
<img src="./img/burlingame_streets.png" width="600" height="300"/></img>
    
* Polygons
    * Structural footprints
    * Boundaries
    
<img src="./img/burlingame.png" width="600" height="300"/></img>

> *Show these in QGIS live*

#### 2.1.2 Raster Data Model
Raster data models are pixel-based representations of continuous surfaces. They are commonly used to model:
* Elevation
* Imagery (brightness values)
* Air quality

Many of the operations available to vector data models are not available to raster data models, but we can use raster data to create features at the areal or zonal level using zonal statistics. Here are some examples:
* Elevation range within a polygon
* Mean vegetation index
* Total population

#### Alternative Data Models
* Network models
    * Nodes and edges
    * Flow Data
    * Friction or speed
* Triangulated Irregular Network (TIN)
    * Commonly used to model elevation
    * Could be used for any raster depiction

## 2.2 Geospatial Data Abstraction Library (GDAL) and OGR
#### 2.2.1 Overview

<img src="osgeo.png" width="200" height="300"/></img>

The Geospatial Data Abstraction Library (GDAL) and the OpenGIS Simple Features Library are C/C++ libraries with actively developed bindings to Java, Python, Perl, and C#. OGR is the portion of the GDAL/OGR library that handles vector data and includes:
* 84 drivers for database and file-based vector formats
* A single vector data model
* Command-line utility programs such as `ogr2ogr`

GDAL/OGR is the underlying library for handling geospatial data formats in a number of python libraries including `fiona`, `geopandas`, and `pysal`.

#### 2.2.2 Installation and Packaging
OGR, which is what we are interested in for this workshop, is part of the osgeo module and is installed with Anaconda by default. It can also be install via conda using:

`conda install gdal`

Or via the Python Package Index:

`pip install gdal`

In [1]:
# GDAL and OGR are in the osgeo package
from osgeo import ogr, gdal
# Check GDAL version
print(gdal.VersionInfo('VERSION_NUM'))
# Access Help
# print(help(ogr))

2010300


#### 2.2.3 Basic Operations
OGR can read, write, and conduct a number of geometric operations on vector geospatial data including:
* Creating geometries from scratch or source
* Iterating and manipulating geometries
* Writing geometries to source

In [2]:
# Create a polygon geometry
ring = ogr.Geometry(ogr.wkbLinearRing)
ring.AddPoint(0.0, 0.0)
ring.AddPoint(0.0,10.0)
ring.AddPoint(10.0,10.0)
ring.AddPoint(10.0,0.0)

poly = ogr.Geometry(ogr.wkbPolygon)
poly.AddGeometry(ring)
print(poly.ExportToWkt())

POLYGON ((0 0 0,0 10 0,10 10 0,10 0 0))


In [3]:
# Iterate over Points in a Polygon
for ring in poly:
    for point in range(0, ring.GetPointCount()):
        # GetPoint returns a tuple not a Geometry
        pt = ring.GetPoint(point)
        print("%i). POINT (%d %d)" %(point, pt[0], pt[1]))

0). POINT (0 0)
1). POINT (0 10)
2). POINT (10 10)
3). POINT (10 0)


In [4]:
# Write a geometry to file
geojson = poly.ExportToJson()
print(geojson)

{ "type": "Polygon", "coordinates": [ [ [ 0.0, 0.0, 0.0 ], [ 0.0, 10.0, 0.0 ], [ 10.0, 10.0, 0.0 ], [ 10.0, 0.0, 0.0 ] ] ] }


## 2.3 File-based Formats
#### 2.3.1 Environmental Systems Research Institute (ESRI) Shapefiles


[Shapefiles](https://www.esri.com/library/whitepapers/pdfs/shapefile.pdf) are arguably the most popular file-based vector geospatial data format. The specification was created by ESRI in the late 1990s as a proprietaty format for storing non-topological vector geometries. Although the format is popular with seasoned GIS Analysts, it has lost favor recently due to a number of limitations including:
* 2GB limitation on component files (shapefile consist of a .shp, .shx, and .dbf file as a minimum)
* Limited support for unicode characters and `null` values represented as zero
* Limited unicode character support

Despite these limitations, shapefiles have endured and much of the open geosptial data on the web is in shapefile format. Luckily, GDAL/OGR has maintained
a shapefile driver for some time.

In [5]:
# Open a shapefile and count the number of features
import os
shapefile = r"cal_housing.shp"
driver = ogr.GetDriverByName('ESRI Shapefile')
cal_housing_shp = driver.Open(shapefile, 0)
layer = cal_housing_shp.GetLayer()
featureCount = layer.GetFeatureCount()
print(featureCount)

20640


#### 2.3.2 Geographic Javascript Object Notation (GeoJSON)
[GeoJSON](http://geojson.org/geojson-spec.html) is vector geosptial data format created in 2008 to address issues with using traditional geosptial data formats in web mapping applications. GeoJSON is based on the popular JSON and consists of a single file that is human readable.

`{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [-82.624, 27.367]
  },
  "properties": {
    "name": "Longboat Key"
  }
}`

Despite the single file format and readability, GeoJSON also suffers from some limitations including:
* Inefficient storage (Cal Housing 10x larger on disk v. shapefile)
* Also lacks topology (although this was partially solved by TopoJSON)
* Not fully supported by all GIS software

In [6]:
# Create OGR geometry from GeoJSON
geojson = r"cal_housing.geojson"
driver = ogr.GetDriverByName('GeoJSON')
cal_housing_gj = driver.Open(geojson, 0)
layer = cal_housing_gj.GetLayer()
featureCount = layer.GetFeatureCount()
print(featureCount)

20640


#### 2.3.3 Keyhole Markup Language (KML)
[Keyhole Markup Language](http://www.opengeospatial.org/standards/kml) is an OGC standard format that was originally created by Keyhole Inc. which was acquired by Google and eventually created Google Earth and Google Maps.

~~## 2.3 Database Formats~~
~~#### 2.3.1 File GeoDatabase~~


## Using fiona to Read Geospatial Data
#### [fiona](http://toblerity.org/fiona/) is a Python wrapper around OGR that:
* Provides more natural bindings to OGR
* Performs well on many geosptial formats
* Uses Python friendly data structures
<img src="./img/cal_housing.png" width="600" height="300"/></img>

In [7]:
import pprint
import fiona
cal_housing = fiona.open("cal_housing.geojson")
cal_housing

<open Collection 'cal_housing.geojson:OGRGeoJSON', mode 'r' at 0x7fe83803f780>

In [8]:
cal_housing.driver

'GeoJSON'

In [9]:
cal_housing.crs

{'init': 'epsg:4326'}

In [10]:
len(cal_housing)

20640

## Using Geopandas to Read Geospatial Data into DataFrames
#### [GeoPandas](http://geopandas.org/) is:
* All-in-one tool for handling geospatial in Python
* Built on top of fiona, shapely, and GDAL/OGR
* User pandas-like data structures such GeoDataFrame and GeoSeries

In [ ]:
import geopandas as gpd
%matplotlib inline
# stpete_parcels.shp is 12,960 observations @ 2,238,504 bytes
parcels = gpd.read_file("./data/stpete_parcels.shp")
parcels.head(3)

,BK,LOT,PARCELID,PARCELID_1,PARCELID_D,PARCELNO,PARCELSUBT,POLY_ID,RG,SB,SC,SHAPE_AREA,SHAPE_LEN,STRAP,TW,geometry
0,000,0430,273116953820000430,27/31/16/95382/000/0430,27-31-16-95382-000-0430,16 31 27 95382 000 0430,0,1,16,95382,27,3999.882294,279.999402,163127953820000430,31,POLYGON ((-82.68981479872396 27.76168992653696...
1,007,0050,183116446400070050,18/31/16/44640/007/0050,18-31-16-44640-007-0050,16 31 18 44640 007 0050,0,2,16,44640,18,9653.587830,400.197524,163118446400070050,31,POLYGON ((-82.74333504227978 27.78891005735331...
2,004,0030,203116669780040030,20/31/16/66978/004/0030,20-31-16-66978-004-0030,16 31 20 66978 004 0030,0,3,16,66978,20,6730.503577,359.979371,163120669780040030,31,POLYGON ((-82.72776976867324 27.76983371840464...


In [ ]:
parcels.plot()

## Reading Geospatial Data Using PySAL

In [ ]:
import pysal
parcels = pysal.open('stpete_parcels.shp')
len(parcels)

> *Introduce the data and motivation for the use case*